In [1]:
import pandas as pd
import numpy as np
dataset=pd.read_csv("ckd.csv")


In [2]:
dataset=pd.get_dummies(dataset, drop_first=True)
dataset.columns

Index(['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes', 'classification_yes'],
      dtype='object')

In [3]:
independent=dataset[['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes']]



In [4]:

dependent=dataset['classification_yes']


In [5]:

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(independent, dependent, test_size = 1/3, random_state = 0)

In [6]:
from sklearn.preprocessing import StandardScaler
SC=StandardScaler()
x_train=SC.fit_transform(x_train)
x_test=SC.transform(x_test)

In [9]:
from sklearn.naive_bayes import ComplementNB
from sklearn.model_selection import GridSearchCV
param_grid= {'alpha': [0.1, 0.5, 1.0, 1.5, 2.0, 5.0, 10.0]}

grid=GridSearchCV(ComplementNB(),param_grid,scoring='f1_weighted')
grid.fit(x_train,y_train)

C:\Users\poorn\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
35 fits failed out of a total of 35.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
35 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\poorn\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\poorn\Anaconda3\lib\site-packages\sklearn\naive_bayes.py", line 690, in fit
    self._count(X, Y)
  File "C:\Users\poorn\Anaconda3\lib\site-packages\sklearn\naive_bayes.py", line 1006, in _count
    check_non_negative(X, "ComplementNB (input X)")
  File "C:\Users\poorn\Anaconda3\lib\site-pack

ValueError: Negative values in data passed to ComplementNB (input X)

In [9]:
# print best parameter after tuning 
#print(grid.best_params_) 


In [10]:
re=grid.cv_results_
print(re)


{'mean_fit_time': array([0.0013411 , 0.00057583, 0.00182281, 0.00098748, 0.        ,
       0.        , 0.00037966]), 'std_fit_time': array([0.00080735, 0.00095181, 0.00316117, 0.00105089, 0.        ,
       0.        , 0.00075932]), 'mean_score_time': array([0.00147953, 0.00349431, 0.00262027, 0.00167375, 0.00163102,
       0.00320363, 0.        ]), 'std_score_time': array([0.00105805, 0.00306444, 0.00173734, 0.0010814 , 0.00326204,
       0.00545885, 0.        ]), 'param_var_smoothing': masked_array(data=[1e-12, 1e-09, 1e-08, 1e-06, 0.0001, 0.01, 0.1],
             mask=[False, False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'params': [{'var_smoothing': 1e-12}, {'var_smoothing': 1e-09}, {'var_smoothing': 1e-08}, {'var_smoothing': 1e-06}, {'var_smoothing': 0.0001}, {'var_smoothing': 0.01}, {'var_smoothing': 0.1}], 'split0_test_score': array([0.98156912, 0.98156912, 0.98156912, 0.98156912, 0.98156912,
       0.98156912, 0.92697811]), 'split1

In [11]:
grid_predictions = grid.predict(x_test) 

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)
cm


array([[51,  0],
       [ 3, 79]], dtype=int64)

In [12]:

# print classification report 
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)
print(clf_report)

              precision    recall  f1-score   support

           0       0.94      1.00      0.97        51
           1       1.00      0.96      0.98        82

    accuracy                           0.98       133
   macro avg       0.97      0.98      0.98       133
weighted avg       0.98      0.98      0.98       133



In [13]:

from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)


The f1_macro value for best parameter {'var_smoothing': 1e-08}: 0.9775556904684072


In [14]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_test,grid.predict_proba(x_test)[:,1])


1.0

In [15]:
table=pd.DataFrame.from_dict(re)

In [16]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_var_smoothing,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.001341,0.000807,0.001480,0.001058,1e-12,{'var_smoothing': 1e-12},0.981569,0.981014,0.962573,1.0,0.981217,0.981275,0.011837,4
1,0.000576,0.000952,0.003494,0.003064,1e-09,{'var_smoothing': 1e-09},0.981569,0.981014,0.962573,1.0,0.981217,0.981275,0.011837,4
2,0.001823,0.003161,0.002620,0.001737,1e-08,{'var_smoothing': 1e-08},0.981569,0.981014,0.981217,1.0,0.981217,0.985003,0.007500,1
3,0.000987,0.001051,0.001674,0.001081,1e-06,{'var_smoothing': 1e-06},0.981569,0.981014,0.981217,1.0,0.981217,0.985003,0.007500,1
4,0.000000,0.000000,0.001631,0.003262,0.0001,{'var_smoothing': 0.0001},0.981569,0.981014,0.981217,1.0,0.981217,0.985003,0.007500,1
5,0.000000,0.000000,0.003204,0.005459,0.01,{'var_smoothing': 0.01},0.981569,0.962264,0.981217,1.0,0.981217,0.981253,0.011934,6
6,0.000380,0.000759,0.000000,0.000000,0.1,{'var_smoothing': 0.1},0.926978,0.962264,0.944023,1.0,0.981217,0.962896,0.025919,7
